In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

## Multiplying Matrices

In [2]:
key = random.PRNGKey(0)
x = random.normal(key, (10,))
print(x)

[-0.3721109   0.26423115 -0.18252768 -0.7368197  -0.44030377 -0.1521442
 -0.67135346 -0.5908641   0.73168886  0.5673026 ]


我们使用block_until_ready是因为JAX默认是[异步执行](https://blog.csdn.net/m0_63003326/article/details/125813341)

In [3]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()  # runs on the GPU

27.9 ms ± 2.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


JAX的numpy函数可以在一般的Numpy数组上工作。

In [4]:
import numpy as np
x = np.random.normal(size=(size, size)).astype(np.float32)
%timeit jnp.dot(x, x.T).block_until_ready()

50.7 ms ± 1.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


比之前慢是因为它每次都将数据转移到GPU上。你可以确保一个NDArray是在设备内存中，使用device_put()。

In [5]:
from jax import device_put

x = np.random.normal(size=(size, size)).astype(np.float32)
x = device_put(x)
%timeit jnp.dot(x, x.T).block_until_ready()

27.3 ms ± 327 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


device_output()的输入仍然是一个NDArray，但是只有当其被使用时才会将数值复制回CPU，其等价于函数jit(lambda x: x)，但是更快。

Jax不只是一个GPU后端的Numpy，它也存在很多写数值计算代码很有用的特征。下面是主要的部分：
+ jit()，加速你的代码。
+ grad()，求导数
+ vmap()，自动向量化

## Using jit() to speed up functions
如果我们有一系列操作，我们可以使用`@jit`装饰器来一起使用XLA(加速线性代数)来编译。

In [6]:
def selu(x, alpha=1.67, lmbda=1.05):
    return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000, ))
%timeit selu(x).block_until_ready()

861 µs ± 385 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


我们可以使用`@jit`来加速，它将会在`selu`被一次调用的时候编译之后存储起来。

In [7]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

142 µs ± 4.72 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Taking derivatives with grad()

In [9]:
def sum_logistic(x):
    return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))
x_small = jnp.arange(3.)
derivative_fn = grad(sum_logistic)
print(derivative_fn(x_small))

[0.25       0.19661194 0.10499357]


检验一下我们的结果是否正确

In [10]:
def first_finite_difference(f, x):
    eps = 1e-3
    return jnp.array([(f(x + eps * v) - f(x - eps * v)) / (2 * eps) for v in jnp.eye(len(x))])
print(first_finite_difference(sum_logistic, x_small))

[0.24998187 0.1965761  0.10502338]


我们可以很容易调用`grad().grad()`和`jit()`，可以被任意混合。我们可以：

In [11]:
grad(jit(grad(jit(grad(sum_logistic)))))(1.0)

DeviceArray(-0.0353256, dtype=float32, weak_type=True)

对于更高级的自动微分，我们可以使用`jax.vjp()`来进行reverse-mode向量雅克比相乘和`jax.jvp()`对于forward-mode 雅克比向量相乘。这两种操作可以互相或者与其它JAX变换结合。下面是一个高效计算海森矩阵的方式：

In [66]:
from jax import jacfwd, jacrev
def hessian(fun):
    return jit(jacfwd(jacrev(fun)))
# 这里jacfwd和jacrrev都会返回雅克比矩阵，两种方式返回结果相同，只不过自动微分实现机制不同

In [67]:
def f(x):
    return jnp.asarray(
        [x[0], 5*x[2], 4*x[1]**2 - 2*x[2], x[2] * jnp.sin(x[0])])

In [68]:
hessian(f)(jnp.array([1.0, 2.0, 3.0]))

DeviceArray([[[ 0.       ,  0.       ,  0.       ],
              [ 0.       ,  0.       ,  0.       ],
              [ 0.       ,  0.       ,  0.       ]],

             [[ 0.       ,  0.       ,  0.       ],
              [ 0.       ,  0.       ,  0.       ],
              [ 0.       ,  0.       ,  0.       ]],

             [[ 0.       ,  0.       ,  0.       ],
              [ 0.       ,  8.       ,  0.       ],
              [ 0.       ,  0.       ,  0.       ]],

             [[-2.524413 ,  0.       ,  0.5403023],
              [ 0.       ,  0.       ,  0.       ],
              [ 0.5403023,  0.       ,  0.       ]]], dtype=float32)

In [69]:
# 关于jax.vjp，grad()被认为是vjp()的特殊情况
import jax
def f(x, y):
    return jax.numpy.sin(x), jax.numpy.cos(y)
primals, f_vjp = jax.vjp(f, 0.5, 1.0)
xbar, ybar = f_vjp((0.5, 1.0))
# 比较底层，没看懂，可能得明白自动微分原理

## Auto-vectorization with vmap()

In [70]:
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))

def apply_matrix(v):
    return jnp.dot(mat, v)

我们可以用循环的方式将batched_x中的每一个元素与mat相乘

In [71]:
def naively_batched_apply_matrix(v_batched):
    return jnp.stack([apply_matrix(v) for v in v_batched])
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

2.54 ms ± 294 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


我们可以让这种操作自动进行

In [72]:
@jit
def batched_apply_matrix(v_batched):
    return jnp.dot(v_batched, mat.T)
%timeit batched_apply_matrix(batched_x).block_until_ready()

26.5 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


可以使用vmap()

In [73]:
@jit
def vmap_batched_apply_matrix(v_batched):
    return vmap(apply_matrix)(v_batched)
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

32.6 µs ± 4.39 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
